In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from models.managers.db_manager import connect_to_bdd

# Connexion à la base de données
conn = connect_to_bdd()

# Requête pour election_2022_t1 et sauvegarde dans df_election
query_election = """
    SELECT winner, code_postal, "Libellé de la commune"
    FROM election_2022_t1
"""
df_election = pd.read_sql(query_election, conn)

# Requête pour economie et sauvegarde dans df_economie
query_economie = """
    SELECT avg_1982, avg_1983, avg_1984, avg_1985, avg_1986, avg_1987, avg_1988, avg_1989,
           avg_1990, avg_1991, avg_1992, avg_1993, avg_1994, avg_1995, avg_1996, avg_1997,
           avg_1998, avg_1999, avg_2000, avg_2001, avg_2002, avg_2003, avg_2004, avg_2005,
           avg_2006, avg_2007, avg_2008, avg_2009, avg_2010, avg_2011, avg_2012, avg_2013,
           avg_2014, avg_2015, avg_2016, avg_2017, avg_2018, avg_2019, avg_2020, avg_2021,
           avg_2022, avg_2023, code_postal
    FROM economie
"""
df_economie = pd.read_sql(query_economie, conn)

# Requête pour securite et sauvegarde dans df_securite
query_securite = """
    SELECT code_postal, tauxpourcent
    FROM securite
"""
df_securite = pd.read_sql(query_securite, conn)

# Requête pour densite et sauvegarde dans df_densite
query_densite = """
    SELECT code_postal, dens_pop
    FROM demographie
"""
df_densite = pd.read_sql(query_densite, conn)


# Fusionner les données
df_combined = pd.merge(df_election, df_economie, on="code_postal")
#df_combined = pd.merge(df_combined, df_securite, on="code_postal")
#df_combined = pd.merge(df_combined, df_densite, on="code_postal")

# Séparer les caractéristiques et la cible
X = df_combined.drop(columns=["winner", "Libellé de la commune"])
y = df_combined["winner"]

# Créer le modèle RandomForestClassifier
clf = RandomForestClassifier()

# Entraîner le modèle sur l'ensemble des données
clf.fit(X, y)

# Requête pour les données de l'Yonne pour les élections
query_election_yonne = """
    SELECT winner, code_postal, "Libellé de la commune"
    FROM election_2022_t1
    WHERE code_postal = '89'
"""
df_election_yonne = pd.read_sql(query_election_yonne, conn)

# Requête pour economie et sauvegarde dans df_economie
query_economie = """
    SELECT avg_1982, avg_1983, avg_1984, avg_1985, avg_1986, avg_1987, avg_1988, avg_1989,
           avg_1990, avg_1991, avg_1992, avg_1993, avg_1994, avg_1995, avg_1996, avg_1997,
           avg_1998, avg_1999, avg_2000, avg_2001, avg_2002, avg_2003, avg_2004, avg_2005,
           avg_2006, avg_2007, avg_2008, avg_2009, avg_2010, avg_2011, avg_2012, avg_2013,
           avg_2014, avg_2015, avg_2016, avg_2017, avg_2018, avg_2019, avg_2020, avg_2021,
           avg_2022, avg_2023, code_postal
    FROM economie
"""
df_economie = pd.read_sql(query_economie, conn)

# Fermeture de la connexion
conn.close()

# Fusionner les données
df_combined = pd.merge(df_election_yonne, df_economie, on="code_postal")

# Séparer les caractéristiques et la cible
X = df_combined.drop(columns=["winner", "Libellé de la commune"])
y = df_combined["winner"]

# Créer le modèle RandomForestClassifier
clf = RandomForestClassifier()

# Entraîner le modèle sur l'ensemble des données
clf.fit(X, y)

# Prédire les probabilités pour chaque classe
y_pred_proba = clf.predict_proba(X)

# Déterminer si chaque parti va dépasser les 5% de votants
parties_depassant_5pct = []
for party_index, party_name in enumerate(clf.classes_):
    proba_threshold = 0.05
    party_proba = y_pred_proba[:, party_index]
    if any(party_proba > proba_threshold):
        parties_depassant_5pct.append((party_name, True))
    else:
        parties_depassant_5pct.append((party_name, False))

# Afficher les résultats
print("Dépassement des 5% pour chaque parti dans l'Yonne :")
for party_name, is_above_5pct in parties_depassant_5pct:
    print(f"Le parti {party_name} va-t-il dépasser les 5% de votants ? : {is_above_5pct}")


/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_3092/3612143465.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election = pd.read_sql(query_election, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_3092/3612143465.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economie = pd.read_sql(query_economie, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_3092/3612143465.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_securite = pd.read_sql(query_securite, conn)
/

Dépassement des 5% pour chaque parti dans l'Yonne :
Le parti La France Insoumise va-t-il dépasser les 5% de votants ? : False
Le parti La République En Marche va-t-il dépasser les 5% de votants ? : True
Le parti Rassemblement National va-t-il dépasser les 5% de votants ? : True


/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_3092/3612143465.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election_yonne = pd.read_sql(query_election_yonne, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_3092/3612143465.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economie = pd.read_sql(query_economie, conn)
